In [ ]:
import numpy as np
import cv2
from scipy import signal
from scipy.fft import fftshift
import os
from scipy.signal import find_peaks,peak_prominences
from scipy.io.wavfile import write
import muDIC as dic
from muDIC import vlab
from PIL import Image
import subprocess

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (0,0), fx = 0.5, fy = 0.5)
            images.append(img)
    return images
    
def highpass_normal_filter_audio(audio, window):
    filtered_audio = None
    filtered_audio = np.zeros(len(audio))
    # compute the mean of each windowed neighborhood
    for i in range(window//2, len(audio) - window//2):
        filtered_audio[i] = np.mean(audio[i-window//2:i+window//2])
    
    filtered_audio = filtered_audio
    filtered_audio[0:window//2] = audio[0:window//2]
    filtered_audio[len(audio) - window//2:] = audio[len(audio) - window//2:]

    return filtered_audio

def lowpass_fft_filter_audio(audio, threshold):
    # transform to frequency domain
    fft = np.fft.fft(audio)
    freq = np.fft.fftfreq(len(fft)) 
    filtered = fft.copy()
    
    # filter by frequency
    filtered = filtered * (abs(freq) < threshold)
    amplitude_filtered = filtered
    
    # transform back to time domain
    filtered_audio = np.fft.ifft(amplitude_filtered)
    filtered_audio = filtered_audio.real
    
    #end_solution
    return filtered_audio

In [ ]:
filenames = ["2_1", "3_1", "4_1", "5_1", "6_1", "7_1"]
save_audio = 1

for filename in filenames:
    shimmer_level = int(filename.split("_")[0])
    frames = []
    frames = load_images_from_folder(filename + '/')

    #-----------------------------------------------------------------------
    # define DIC patches
    import logging
    logger = logging.getLogger()
    logger.setLevel(logging.CRITICAL)

    fields_samples = []
    patch_size = 75

    start_x = [135, 230, 330, 430, 135, 230, 330, 430, 135, 230, 330, 430]
    start_y = [47, 47, 47, 47, 142, 142, 142, 142, 237, 237, 237, 237]
    num_patches = len(start_x)

    #-----------------------------------------------------------------------
    # Perform DIC for all patches
    for patch_num in range(num_patches):
        fields_stored = []
        for idx in range(len(frames) - 1):
            frame = frames[idx + 1]
            ref = frames[idx]
            
            # load reference and current images, then perform a gaussian blur
            image_stack = dic.image_stack_from_list([ref, frame])
            image_stack.set_filter(dic.filtering.lowpass_gaussian,sigma=1.0)
            
            # create the distortion mesh control points
            mesher = dic.Mesher()
            mesh = mesher.mesh(image_stack, start_x[patch_num], start_x[patch_num] + patch_size, start_y[patch_num], start_y[patch_num] + patch_size, 5, 5, GUI=False)
            
            # run DIC
            inputs = dic.DICInput(mesh,image_stack, ref_update_frames=[0], pad=20)
            dic_job = dic.DICAnalysis(inputs)
            results = dic_job.run()
            
            # store results
            fields = dic.Fields(results)
            fields_stored.append(fields)            

        print(patch_num + 1, "out of", num_patches)
        fields_samples.append(fields_stored)

    logger.setLevel(logging.DEBUG)

    #-----------------------------------------------------------------------
    # compute the distortion magnitudes
    disps_samples = []
    for field_sample in fields_samples:
        disps = []
        
        # merge u and v displacements
        for fields in field_sample:
            disps.append(np.sum((fields.disp()[:, 0, :, :, 1] ** 2 + fields.disp()[:, 1, :, :, 1] ** 2)))
        disps_samples.append(disps)
    
    # check for outliers 
    for disps_sample in disps_samples:
        for idx in range(len(disps_sample)):
            if disps_sample[idx] > 20:
                disps_sample[idx] = disps_sample[idx-1]

    # store results
    disp_result = np.zeros(300)
    for disps_sample in disps_samples:
        disp_result += disps_sample

    disp_result = disp_result / num_patches
    #-----------------------------------------------------------------------
    # Run the moving window filter
    if shimmer_level != 6 and shimmer_level != 7:
        window_size = 3
        if shimmer_level == 2 or shimmer_level == 3:
            window_size = 4

        filtered_audio = highpass_normal_filter_audio(disp_result,window_size)
    #-----------------------------------------------------------------------
    # Run the low pass filter
    if shimmer_level == 2:
        threshold = 0.14
    if shimmer_level == 3:
        threshold = 0.155
    if shimmer_level == 4:
        threshold = 0.155
    if shimmer_level == 5:
        threshold = 0.13
    if shimmer_level == 6:
        threshold = 0.14
    if shimmer_level == 7:
        threshold = 0.15
        
    if shimmer_level == 6 or shimmer_level == 7:
        filtered_audio = lowpass_fft_filter_audio(disp_result, threshold)
    else:
        filtered_audio = lowpass_fft_filter_audio(filtered_audio, threshold)
    #-----------------------------------------------------------------------
    # find peaks of the smoothed shimmer displacement frequency
    if shimmer_level == 2:
        height_setting = np.mean(filtered_audio) * 1.05
    if shimmer_level == 3:
        height_setting = 0.081
    if shimmer_level == 4:
        height_setting = 0.088
    if shimmer_level == 5:
        height_setting = 0.1
    if shimmer_level == 6:
        height_setting = 0.1
    if shimmer_level == 7:
        height_setting = 0.1

    peaks, _ = find_peaks(filtered_audio, height=height_setting)
    if np.sign(np.diff(filtered_audio))[0] < 0:
        peaks = np.insert(peaks, 0, 0)
    if np.sign(np.diff(filtered_audio))[-1] > 0:
        peaks = np.insert(peaks, len(peaks), len(filtered_audio)-1)
    #-----------------------------------------------------------------------
    # generate audio
    if save_audio:
        #-----------------------------------------------------------------------
        # in phase audio
        sampleRate = 44100
        frequency = 3500
        length = 0.007

        # create sine tone
        t = np.linspace(0, length, int(sampleRate * length))
        y = np.sin(frequency * 2 * np.pi * t)
        amplitude = np.iinfo(np.int16).max
        data = y
        
        # place sine tones at shimmer peaks
        audio_arr = np.zeros(5 * 44100)
        for peak in peaks:
            start = int(peak * (5 * 44100 / 300))
            end = start + int(0.007 * 44100)
            audio_arr[start:end] = data

        # save audio
        rate = 44100
        scaled = np.int16(audio_arr / np.max(np.abs(audio_arr)) * 32767)
        write('in_phase.wav', rate, scaled)
        
        # merge with video
        command = f"ffmpeg -i unity_shimmers/{filename}.mp4 -i in_phase.wav -c:v copy -c:a aac unity_shimmers/{filename}_in_phase.mp4"
        subprocess.call(command, shell=True)
        #-----------------------------------------------------------------------
        # fully out of phase audio
        for i in range(1, 6):
            sampleRate = 44100
            frequency = 3500
            length = 0.007

            # create sine tone
            t = np.linspace(0, length, int(sampleRate * length))
            y = np.sin(frequency * 2 * np.pi * t)
            amplitude = np.iinfo(np.int16).max
            data = y

            # place sine tones out of phase
            audio_arr = np.zeros(5 * 44100)
            for peak in peaks:
                # 23ms to 80ms interval, after the end of current frame, within ~100ms window
                start = int(peak * (5 * 44100 / 300)) + int(np.random.uniform(0.023, 0.080, 1)  * 44100) + int(1 / 60  * 44100)
                end = start + int(0.007 * 44100)
                if end < len(audio_arr):
                    audio_arr[start:end] = data
            
            # save audio
            rate = 44100
            scaled = np.int16(audio_arr / np.max(np.abs(audio_arr)) * 32767)
            write('out_phase.wav', rate, scaled)
            
            # merge with video
            command = f"ffmpeg -i unity_shimmers/{filename}.mp4 -i out_phase.wav -c:v copy -c:a aac unity_shimmers/{filename}_out_phase_{i}.mp4"
            subprocess.call(command, shell=True)
        #-----------------------------------------------------------------------
        # half out of phase audio
        for i in range(1, 6):
            sampleRate = 44100
            frequency = 3500
            length = 0.007

            # create sine tone
            t = np.linspace(0, length, int(sampleRate * length))
            y = np.sin(frequency * 2 * np.pi * t)
            amplitude = np.iinfo(np.int16).max
            data = y

            audio_arr = np.zeros(5 * 44100)

            # place sine tones out of phase, however only keep half of the total tones
            for peak in peaks:
                if np.random.rand(1) > 0.5:
                    start = int(peak * (5 * 44100 / 300)) + int(np.random.uniform(0.023, 0.080, 1)  * 44100) + int(1 / 60  * 44100)
                    end = start + int(0.007 * 44100)
                    if end < len(audio_arr):
                        audio_arr[start:end] = data

            # save audio
            rate = 44100
            scaled = np.int16(audio_arr / np.max(np.abs(audio_arr)) * 32767)
            write('half_phase.wav', rate, scaled)
            
            # merge with video
            command = f"ffmpeg -i unity_shimmers/{filename}.mp4 -i half_phase.wav -c:v copy -c:a aac unity_shimmers/{filename}_half_phase_{i}.mp4"
            subprocess.call(command, shell=True)

        #-----------------------------------------------------------------------